In [47]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pathlib
import time
import contextily as ctx # for basemaps
import os
import datetime
%matplotlib inline

In [2]:
# turn response caching on and turn on logging to your terminal window
ox.config(log_console=True, use_cache=True)

ox.__version__

'1.0.1'

## Read Census data

In [5]:
mexico_censo = pd.read_csv('input_data/censo_mexico_2020_csv/iter_00_cpv2020/conjunto_de_datos/conjunto_de_datos_iter_00CSV20.csv', encoding='utf-8', low_memory=False)

In [6]:
mexico_censo.columns

Index(['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT',
       ...
       'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ',
       'VPH_SINRTV', 'VPH_SINLTC', 'VPH_SINCINT', 'VPH_SINTIC', 'TAMLOC'],
      dtype='object', length=232)

In [7]:
# get data of cdmx as this is a special case (review later)
cdmx = mexico_censo[mexico_censo['ENTIDAD'] == 9]

In [8]:
# drop the different totals there are in the dataframe, which may affect the results when sorting by population
mexico_censo.drop(mexico_censo[(mexico_censo['LOC'] == 0) | (mexico_censo['LOC'] == 9998) | (mexico_censo['LOC'] == 9999)].index, inplace=True)
mexico_censo.reset_index(inplace=True, drop=True)

In [9]:
# select only some columns
mexico_censo = mexico_censo[['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'POBTOT']]

In [10]:
# sort by population to find the most populated cities
mexico_censo_sorted_by_population = mexico_censo.sort_values(by=['POBTOT'], ascending=False)

In [11]:
# get the first 100 most populated cities
mexico_censo_sorted_by_population_top100 = mexico_censo_sorted_by_population.head(100)

In [12]:
mexico_censo_sorted_by_population_top100.reset_index(inplace=True, drop=True)

## Read Marco Geoestadistico data

In [13]:
mexico_MG = gpd.read_file('input_data/marco_geoestadistico_2020/MG_2020_Integrado/conjunto_de_datos/00l.shp')

## Prepare data for merging

In [14]:
# We need to add some zeros to the values on the elements from the CENSO 2020 dataframe that conform the CVEGEO.
#data.loc[:, ['ENTIDAD']] = data.loc[:, ['ENTIDAD']].applymap('{0:0>2}'.format)
#data.loc[:, ['MUN']] = data.loc[:, ['MUN']].applymap('{0:0>3}'.format)
#data.loc[:, ['LOC']] = data.loc[:, ['LOC']].applymap('{0:0>4}'.format)
mexico_censo_sorted_by_population_top100['ENTIDAD'] = mexico_censo_sorted_by_population_top100['ENTIDAD'].apply(lambda x: '{0:0>2}'.format(x))
mexico_censo_sorted_by_population_top100['MUN'] = mexico_censo_sorted_by_population_top100['MUN'].apply(lambda x: '{0:0>3}'.format(x))
mexico_censo_sorted_by_population_top100['LOC'] = mexico_censo_sorted_by_population_top100['LOC'].apply(lambda x: '{0:0>4}'.format(x))

<ipython-input-14-ef1874bbc4ea>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mexico_censo_sorted_by_population_top100['ENTIDAD'] = mexico_censo_sorted_by_population_top100['ENTIDAD'].apply(lambda x: '{0:0>2}'.format(x))
<ipython-input-14-ef1874bbc4ea>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mexico_censo_sorted_by_population_top100['MUN'] = mexico_censo_sorted_by_population_top100['MUN'].apply(lambda x: '{0:0>3}'.format(x))
<ipython-input-14-ef1874bbc4ea>:7: SettingWithCopyWarning: 
A value is t

In [15]:
# convert column values into strings
mexico_censo_sorted_by_population_top100 = mexico_censo_sorted_by_population_top100.astype({'ENTIDAD':str, 'MUN':str, 'LOC':str})

In [16]:
# Concatenate columns to create the CVEGEO columns
mexico_censo_sorted_by_population_top100['CVEGEO'] = mexico_censo_sorted_by_population_top100['ENTIDAD'] + mexico_censo_sorted_by_population_top100['MUN'] + mexico_censo_sorted_by_population_top100['LOC']
mexico_censo_sorted_by_population_top100.reset_index(inplace=True, drop=True)

## Merge dataframes

In [17]:
#yucatan_ageb_marcogeo2020.sort_values(by=['CVEGEO'], ignore_index=True, inplace=True)

#drop columns that are duplicated in the other dataframe
mexico_censo_sorted_by_population_top100.drop(columns=['ENTIDAD', 'MUN', 'LOC'], inplace = True)
mexico_MG.drop(columns=['NOMGEO', 'AMBITO'], inplace=True)

mexico_pop_top100 = mexico_MG.merge(mexico_censo_sorted_by_population_top100, on=['CVEGEO'], how='inner')

In [18]:
mexico_pop_top100 = mexico_pop_top100.sort_values(by=['POBTOT'], ascending=False)
mexico_pop_top100.reset_index(inplace=True, drop=True)

In [19]:
mexico_pop_top100

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,geometry,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT
0,090070001,09,007,0001,"POLYGON ((2808094.185 825698.592, 2808093.949 ...",Ciudad de México,Iztapalapa,Iztapalapa,1835486
1,020040001,02,004,0001,"MULTIPOLYGON (((1086114.495 2332537.239, 10861...",Baja California,Tijuana,Tijuana,1810645
2,150330001,15,033,0001,"POLYGON ((2807108.766 854056.556, 2807126.042 ...",México,Ecatepec de Morelos,Ecatepec de Morelos,1643623
3,110200001,11,020,0001,"MULTIPOLYGON (((2541248.094 1004738.337, 25414...",Guanajuato,León,León de los Aldama,1579803
4,211140001,21,114,0001,"POLYGON ((2896301.113 798603.947, 2896323.425 ...",Puebla,Puebla,Heroica Puebla de Zaragoza,1542232
...,...,...,...,...,...,...,...,...,...
95,070780001,07,078,0001,"POLYGON ((3498959.931 562703.503, 3499000.074 ...",Chiapas,San Cristóbal de las Casas,San Cristóbal de las Casas,183509
96,301310001,30,131,0001,"POLYGON ((2974612.984 959763.024, 2974928.771 ...",Veracruz de Ignacio de la Llave,Poza Rica de Hidalgo,Poza Rica de Hidalgo,180057
97,220160001,22,016,0001,"MULTIPOLYGON (((2711463.894 929406.954, 271156...",Querétaro,San Juan del Río,San Juan del Río,177719
98,260550001,26,055,0001,"POLYGON ((1290014.227 2323361.153, 1290965.908...",Sonora,San Luis Río Colorado,San Luis Río Colorado,176685


## Save as shapefiles

In [20]:
# save data to shapefiles
folder = 'mexico_top100_cities_by_population_shp'
filepath = pathlib.Path(folder) / "mexico_top100_cities_by_population.shp"
# if save folder does not already exist, create it
filepath.parent.mkdir(parents=True, exist_ok=True)
mexico_pop_top100.to_file(filepath)

## Load saved shapefiles

In [22]:
top100 = gpd.read_file('mexico_top100_cities_by_population_shp/mexico_top100_cities_by_population.shp')

## Prepare queries for downloading graphs of the cities

In [24]:
gdf = top100

In [27]:
# project it for CRS 3857 to have it in meter units
gdf = gdf.to_crs(epsg=3857)

In [28]:
# get area of each city, in meters (as it is already in a CRS with meter unit)
def get_area(geometry):
    return geometry.area

gdf['area_m'] = gdf['geometry'].map(get_area)

In [30]:
# project it from CRS 3857 to 4326 for OSM
gdf = gdf.to_crs(epsg=4326)

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [39]:
# where to save networks
gdf['entidad_folder'] = gdf.apply(lambda row: '{}_{}'.format(row['CVE_ENT'], row['NOM_ENT']).replace(' ', '-'), axis=1)
gdf['municipio_folder'] = gdf.apply(lambda row: '{}_{}'.format(row['CVE_MUN'], row['NOM_MUN']).replace(' ', '-'), axis=1)
gdf['localidad_folder'] = gdf.apply(lambda row: '{}/{}/{}_{}'.format(row['entidad_folder'],
                                                                     row['municipio_folder'],
                                                                     row['CVE_LOC'], row['NOM_LOC']).replace(' ', '-'), axis=1)

In [40]:
# create list of queries
queries = gdf.apply(lambda row: {'localidad_folder':row['localidad_folder'], 
                                 'geometry':row['geometry'],
                                 'area_m':row['area_m'],
                                 'POBTOT':row['POBTOT']}, axis=1).tolist()
queries[0]

{'localidad_folder': '09_Ciudad-de-México/007_Iztapalapa/0001_Iztapalapa',
 'geometry': <shapely.geometry.polygon.Polygon at 0x7fce4fe6f0a0>,
 'area_m': 127684813.66466376,
 'POBTOT': 1835486}

## Download road network graphs from polygons of the cities

In [43]:
output_folder = './mexico_cities_graphs/' #where to save graph shapefiles and graphml

In [44]:
network_type = 'drive'
retain_all = True
buffer = False

In [48]:
start_time = time.time()
for query in queries:
    try:
        # load graph and save it if it hasn't already been saved in the output_path
        if not os.path.exists('{}/{}.graphml'.format(output_folder, query['localidad_folder'])):
            geometry = query['geometry'].buffer(0) #fix trivially invalid geometries (nested shells, ring self-intersections)
            G = ox.graph_from_polygon(polygon=geometry, network_type=network_type, retain_all=retain_all)
            G.graph['name'] = query['localidad_folder']
            G.graph['area_m'] = query['area_m']
            G.graph['POBTOT'] = query['POBTOT']
            ox.save_graph_shapefile(G, filepath=output_folder+query['localidad_folder'], directed=True)
            ox.save_graphml(G, filepath=output_folder+'{}.graphml'.format(query['localidad_folder']))
    except Exception as e:
        print('"{}" failed: {}'.format(query['localidad_folder'], e))
print('Finished making graphs in {:,.2f} seconds'.format(time.time()-start_time))

/home/alejandro/anaconda3/envs/road-network-project/lib/python3.8/site-packages/osmnx/io.py:108: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(filepath_nodes, driver="ESRI Shapefile", index=True, encoding=encoding)


Finished making graphs in 3,248.62 seconds


## Make a DataFrame of all the cities that have .graphml files saved in the folder

In [49]:
data_folder = 'mexico_cities_graphs'

In [52]:
places = []
for entidad_folder in os.listdir(data_folder):
    for municipio_folder in os.listdir('{}/{}'.format(data_folder, entidad_folder)):
        for localidad_file in os.listdir('{}/{}/{}'.format(data_folder, entidad_folder, municipio_folder)):
                if '.graphml' in localidad_file:
                    data = {}
                    data['CVE_ENT'] = entidad_folder.split('_')[0]
                    data['NOM_ENT'] = entidad_folder.replace('{}_'.format(data['CVE_ENT']), '').replace('-', ' ')
                    data['CVE_MUN'] = municipio_folder.split('_')[0]
                    data['NOM_MUN'] = municipio_folder.replace('{}_'.format(data['CVE_MUN']), '').replace('-', ' ')
                    data['CVE_LOC'] = localidad_file.split('_')[0]
                    #data['town'] = town_folder.split('_')[1]
                    data['NOM_LOC'] = localidad_file.replace('{}_'.format(data['CVE_LOC']), '').replace('.graphml', '').replace('-', ' ')
                    data['CVEGEO'] = data['CVE_ENT']+data['CVE_MUN']+data['CVE_LOC']
                    data['path'] = '{}/{}/{}'.format(data_folder, entidad_folder, municipio_folder)
                    data['file'] = localidad_file
                    places.append(data)

df = pd.DataFrame(places)

In [53]:
df

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,CVEGEO,path,file
0,26,Sonora,018,Cajeme,0001,Ciudad Obregón,260180001,mexico_cities_graphs/26_Sonora/018_Cajeme,0001_Ciudad-Obregón.graphml
1,26,Sonora,030,Hermosillo,0001,Hermosillo,260300001,mexico_cities_graphs/26_Sonora/030_Hermosillo,0001_Hermosillo.graphml
2,26,Sonora,055,San Luis Río Colorado,0001,San Luis Río Colorado,260550001,mexico_cities_graphs/26_Sonora/055_San-Luis-Rí...,0001_San-Luis-Río-Colorado.graphml
3,26,Sonora,043,Nogales,0001,Heroica Nogales,260430001,mexico_cities_graphs/26_Sonora/043_Nogales,0001_Heroica-Nogales.graphml
4,22,Querétaro,016,San Juan del Río,0001,San Juan del Río,220160001,mexico_cities_graphs/22_Querétaro/016_San-Juan...,0001_San-Juan-del-Río.graphml
...,...,...,...,...,...,...,...,...,...
95,15,México,104,Tlalnepantla de Baz,0001,Tlalnepantla,151040001,mexico_cities_graphs/15_México/104_Tlalnepantl...,0001_Tlalnepantla.graphml
96,15,México,106,Toluca,0001,Toluca de Lerdo,151060001,mexico_cities_graphs/15_México/106_Toluca,0001_Toluca-de-Lerdo.graphml
97,15,México,060,Nicolás Romero,0001,Ciudad Nicolás Romero,150600001,mexico_cities_graphs/15_México/060_Nicolás-Romero,0001_Ciudad-Nicolás-Romero.graphml
98,21,Puebla,114,Puebla,0001,Heroica Puebla de Zaragoza,211140001,mexico_cities_graphs/21_Puebla/114_Puebla,0001_Heroica-Puebla-de-Zaragoza.graphml


### Add total population for sorting and calculation of stats in descendent order

In [56]:
pobtot = gdf[['CVEGEO', 'POBTOT']]

In [59]:
df = df.merge(pobtot, on=['CVEGEO'], how='inner')
df = df_pob.sort_values(by=['POBTOT'], ascending=False)
df.reset_index(inplace=True, drop=True)

## Load graph and calculate stats for each city

In [92]:
def load_graph_get_stats(row):
    
    try:
        start_time = time.time()
        print('START {}/{}'.format(row['path'], row['file']))
        G = ox.load_graphml(filepath=row['path']+'/'+row['file'])
        G = ox.project_graph(G, to_crs=3857) # project graph to CRS pseudo-UTM with meter units
        area_m = float(G.graph['area_m'])
        
        # calculate basic stats
        stats = ox.basic_stats(G, area=area_m, clean_intersects=True, circuity_dist='euclidean')
        
        stats['CVEGEO'] = row['CVEGEO']
        stats['CVE_ENT'] = row['CVE_ENT']
        stats['NOM_ENT'] = row['NOM_ENT']
        stats['CVE_MUN'] = row['CVE_MUN']
        stats['NOM_MUN'] = row['NOM_MUN']
        stats['CVE_LOC'] = row['CVE_LOC']
        stats['NOM_LOC'] = row['NOM_LOC']
        
        # calculate extended stats excluding shortest paths length, eccentricity, diameter, radius, center
        
        # Convert MultiDiGraph to DiGraph.
        # Chooses between parallel edges by minimizing weight attribute value.
        D = ox.utils_graph.get_digraph(G, weight="length")
        # create undirected Graph from the DiGraph, for those metrics that need it
        Gu = nx.Graph(D)
        
         # average degree of the neighborhood of each node, and average for graph
        avg_neighbor_degree = nx.average_neighbor_degree(G)
        stats["avg_neighbor_degree"] = avg_neighbor_degree
        stats["avg_neighbor_degree_avg"] = sum(avg_neighbor_degree.values()) / len(avg_neighbor_degree)
        
        # avg weighted degree of neighborhood of each node, and average for graph
        avg_wtd_nbr_deg = nx.average_neighbor_degree(G, weight="length")
        stats["avg_weighted_neighbor_degree"] = avg_wtd_nbr_deg
        stats["avg_weighted_neighbor_degree_avg"] = sum(avg_wtd_nbr_deg.values()) / len(avg_wtd_nbr_deg)
        
        # degree centrality for a node is the fraction of nodes it is connected to
        degree_centrality = nx.degree_centrality(G)
        stats["degree_centrality"] = degree_centrality
        stats["degree_centrality_avg"] = sum(degree_centrality.values()) / len(degree_centrality)
        
        # calculate clustering coefficient for the nodes
        stats["clustering_coefficient"] = nx.clustering(Gu)
        
        # average clustering coefficient for the graph
        stats["clustering_coefficient_avg"] = nx.average_clustering(Gu)
        
        # calculate weighted clustering coefficient for the nodes
        stats["clustering_coefficient_weighted"] = nx.clustering(Gu, weight="length")
        
        # average clustering coefficient (weighted) for the graph
        stats["clustering_coefficient_weighted_avg"] = nx.average_clustering(Gu, weight="length")
        
        # pagerank: a ranking of the nodes in the graph based on the structure of
        # the incoming links
        pagerank = nx.pagerank(D, weight="length")
        stats["pagerank"] = pagerank
        
        # node with the highest page rank, and its value
        pagerank_max_node = max(pagerank, key=lambda x: pagerank[x])
        stats["pagerank_max_node"] = pagerank_max_node
        stats["pagerank_max"] = pagerank[pagerank_max_node]
        
        # node with the lowest page rank, and its value
        pagerank_min_node = min(pagerank, key=lambda x: pagerank[x])
        stats["pagerank_min_node"] = pagerank_min_node
        stats["pagerank_min"] = pagerank[pagerank_min_node]
        
        # betweenness centrality of a node is the sum of the fraction of
        # all-pairs shortest paths that pass through node. nx2.4+
        # implementation cannot run on Multi(Di)Graphs, so use DiGraph
        btwn_cent = nx.betweenness_centrality(D, weight="length")
        stats["betweenness_centrality"] = btwn_cent
        stats["betweenness_centrality_avg"] = sum(btwn_cent.values()) / len(btwn_cent)
        
        # closeness centrality of a node is the reciprocal of the sum of the
        # shortest path distances from u to all other nodes
        close_cent = nx.closeness_centrality(G, distance="length")
        stats["closeness_centrality"] = close_cent
        stats["closeness_centrality_avg"] = sum(close_cent.values()) / len(close_cent)
        
        # extract nodes of the largest connected component 
        node_list = [node for node in close_cent]
        stats['nodes'] = node_list
                
        stats['area_km'] = area_m / 1e6
        stats['area'] = area_m
        stats['time'] = time.time()-start_time
        print('FINISH {}/{} in {} seconds'.format(row['path'], row['file'], stats['time']))
        return pd.Series(stats)

    except Exception as e:
        print('{}/{} failed: {}'.format(row['path'], row['file'], e))
        return pd.Series()

In [95]:
# first 50
df50 = df.head(50)

In [ ]:
%%time
stats = df50.apply(load_graph_get_stats, axis=1)

START mexico_cities_graphs/09_Ciudad-de-México/007_Iztapalapa/0001_Iztapalapa.graphml
FINISH mexico_cities_graphs/09_Ciudad-de-México/007_Iztapalapa/0001_Iztapalapa.graphml in 10056.798384904861 seconds
START mexico_cities_graphs/02_Baja-California/004_Tijuana/0001_Tijuana.graphml


In [96]:
df50

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,CVEGEO,path,file,POBTOT
0,09,Ciudad de México,007,Iztapalapa,0001,Iztapalapa,090070001,mexico_cities_graphs/09_Ciudad-de-México/007_I...,0001_Iztapalapa.graphml,1835486
1,02,Baja California,004,Tijuana,0001,Tijuana,020040001,mexico_cities_graphs/02_Baja-California/004_Ti...,0001_Tijuana.graphml,1810645
2,15,México,033,Ecatepec de Morelos,0001,Ecatepec de Morelos,150330001,mexico_cities_graphs/15_México/033_Ecatepec-de...,0001_Ecatepec-de-Morelos.graphml,1643623
3,11,Guanajuato,020,León,0001,León de los Aldama,110200001,mexico_cities_graphs/11_Guanajuato/020_León,0001_León-de-los-Aldama.graphml,1579803
4,21,Puebla,114,Puebla,0001,Heroica Puebla de Zaragoza,211140001,mexico_cities_graphs/21_Puebla/114_Puebla,0001_Heroica-Puebla-de-Zaragoza.graphml,1542232
5,08,Chihuahua,037,Juárez,0001,Juárez,080370001,mexico_cities_graphs/08_Chihuahua/037_Juárez,0001_Juárez.graphml,1501551
6,14,Jalisco,039,Guadalajara,0001,Guadalajara,140390001,mexico_cities_graphs/14_Jalisco/039_Guadalajara,0001_Guadalajara.graphml,1385621
7,14,Jalisco,120,Zapopan,0001,Zapopan,141200001,mexico_cities_graphs/14_Jalisco/120_Zapopan,0001_Zapopan.graphml,1257547
8,09,Ciudad de México,005,Gustavo A. Madero,0001,Gustavo A. Madero,090050001,mexico_cities_graphs/09_Ciudad-de-México/005_G...,0001_Gustavo-A.-Madero.graphml,1173351
9,19,Nuevo León,039,Monterrey,0001,Monterrey,190390001,mexico_cities_graphs/19_Nuevo-León/039_Monterrey,0001_Monterrey.graphml,1142952


In [89]:
df.tail(50)

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,CVEGEO,path,file,POBTOT
50,09,Ciudad de México,016,Miguel Hidalgo,0001,Miguel Hidalgo,090160001,mexico_cities_graphs/09_Ciudad-de-México/016_M...,0001_Miguel-Hidalgo.graphml,414470
51,19,Nuevo León,046,San Nicolás de los Garza,0001,San Nicolás de los Garza,190460001,mexico_cities_graphs/19_Nuevo-León/046_San-Nic...,0001_San-Nicolás-de-los-Garza.graphml,412199
52,30,Veracruz de Ignacio de la Llave,193,Veracruz,0001,Veracruz,301930001,mexico_cities_graphs/30_Veracruz-de-Ignacio-de...,0001_Veracruz.graphml,405952
53,09,Ciudad de México,006,Iztacalco,0001,Iztacalco,090060001,mexico_cities_graphs/09_Ciudad-de-México/006_I...,0001_Iztacalco.graphml,404695
54,15,México,081,Tecámac,0019,Ojo de Agua,150810019,mexico_cities_graphs/15_México/081_Tecámac,0019_Ojo-de-Agua.graphml,386290
55,15,México,122,Valle de Chalco Solidaridad,0001,Xico,151220001,mexico_cities_graphs/15_México/122_Valle-de-Ch...,0001_Xico.graphml,384327
56,11,Guanajuato,007,Celaya,0001,Celaya,110070001,mexico_cities_graphs/11_Guanajuato/007_Celaya,0001_Celaya.graphml,378143
57,18,Nayarit,017,Tepic,0001,Tepic,180170001,mexico_cities_graphs/18_Nayarit/017_Tepic,0001_Tepic.graphml,371387
58,15,México,039,Ixtapaluca,0001,Ixtapaluca,150390001,mexico_cities_graphs/15_México/039_Ixtapaluca,0001_Ixtapaluca.graphml,368585
59,17,Morelos,007,Cuernavaca,0001,Cuernavaca,170070001,mexico_cities_graphs/17_Morelos/007_Cuernavaca,0001_Cuernavaca.graphml,341029
